In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

Intitializing Scala interpreter ...

Spark Web UI available at http://5e7a77d7892b:4042
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1596791606374)
SparkSession available as 'spark'


import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 3.0.0


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = /home/jovyan/data/events-stream
modelPath: String = /home/jovyan/models/spark-ml-model
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [14]:
val model = PipelineModel.load(modelPath)

import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

val predictionsDF = model.transform(inputDF)

val predictionsDF2 = predictionsDF.select(predictionsDF.col("*"), getProbability($"probability").alias("clean_probability"))

model: org.apache.spark.ml.PipelineModel = pipeline_50506bd299de
import org.apache.spark.sql.functions._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4425/0x0000000841702840@30a4b46c,DoubleType,List(Some(class[value[0]: vector])),None,false,true)
predictionsDF: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 4 more fields]
predictionsDF2: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 5 more fields]


In [15]:
val task = predictionsDF2.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}
task.start()

task: org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row] = org.apache.spark.sql.streaming.DataStreamWriter@7da579d5
res4: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3035d028


+-------------------+
|  clean_probability|
+-------------------+
| 0.4268717919279794|
| 0.4490940961188765|
|0.45003896436106006|
| 0.5009534496720379|
| 0.5115910332738594|
|0.47990273435412273|
|  0.522451345978401|
| 0.4613909338534102|
| 0.4915041120214799|
|  0.427099851279561|
| 0.5122734885754113|
| 0.5299383040768293|
| 0.4721380497360485|
| 0.5121171935283098|
| 0.4915041120214799|
|  0.534268212134474|
+-------------------+

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|i miss the babies.  |[i, miss, the, ba...|(1000,[17,71,756,...|[5.73128208072020...|[0.57312820807202...|       0.0|
|you're my summer ...|[you're, my, summ...|(1000,[17,71,338,...|[5.50905903881123...|[0


+-------------------+
|  clean_probability|
+-------------------+
| 0.5115910332738594|
| 0.5204172606179233|
| 0.5115910332738594|
| 0.4954681213077484|
|  0.496145960562502|
|0.49242153272303674|
| 0.5447009635419148|
| 0.4548375549037176|
| 0.5292558908305575|
| 0.4894498007866062|
| 0.5447009635419148|
| 0.5115910332738594|
| 0.5450529233977118|
| 0.5115910332738594|
| 0.4871685828374949|
| 0.4915041120214799|
| 0.4915041120214799|
+-------------------+

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Rahm said, &quot;...|[rahm, said,, &qu...|(1000,[17,85,125,...|[4.88408966726140...|[0.48840896672614...|       1.0|
|listening to para...|[listening, to, p...|(1000,[2,10,80,43...|


+-------------------+
|  clean_probability|
+-------------------+
| 0.5115910332738594|
|  0.479267574769894|
| 0.5436019667400642|
| 0.5156593830975428|
| 0.4915041120214799|
| 0.5222326473138066|
|0.47990273435412273|
| 0.4814179407999105|
| 0.4871685828374949|
| 0.4915041120214799|
| 0.5559896080155003|
| 0.4806164859443215|
| 0.5222326473138066|
+-------------------+

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Toad Suck Daze is...|[toad, suck, daze...|(1000,[17,29,80,2...|[4.88408966726140...|[0.48840896672614...|       1.0|
|@BayShoreCC in th...|[@bayshorecc, in,...|(1000,[17,25,36,1...|[5.20732425230106...|[0.52073242523010...|       0.0|
|My &quot;Life is ...|[my, &quot;l


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|@RedMummy Exposin...|[@redmummy, expos...|(1000,[17,80,103,...|[4.88408966726140...|[0.48840896672614...|       1.0|
|@mu5icrage  no go...|[@mu5icrage, , no...|(1000,[79,150,251...|[6.22852302469116...|[0.62285230246911...|       0.0|
|Farewell Kwai Cha...|[farewell, kwai, ...|(1000,[116,321,35...|[4.79053289137720...|[0.47905328913772...|       1.0|
|omg margaret is l...|[omg, margaret, i...|(1000,[17,22,48,1...|[4.88408966726140...|[0.48840896672614...|       1.0|
|Someone tried new...|[someone, tried, ...|(1000,[11,12,150,...|[5.13325849211000...|[0.51332584921100...|       0.0|
|5 am roll call fr...|[5, am, roll, cal...|(1000,[17,48

In [17]:
task.cancel()

<console>: 40: error: value cancel is not a member of org.apache.spark.sql.streaming.DataStreamWriter[org.apache.spark.sql.Row]